# spaCy Entity Extraction

In this notebook we will be looking at using spaCy (https://spacy.io/) to populate object models from patent claim data.

In [1]:
#Let's import spaCy
import spacy

nlp = spacy.load('en') 

### Entity Extraction

For reference here are some common object POS patterns as extracted from a patent specification using the reference numeral as an end point.
```
[('<DET><NOUN><NUM>', 63),
 ('<DET><NOUN><NOUN><NUM>', 50),
 ('<DET><VERB><NOUN><NUM>', 48),
 ('<DET><ADJ><NOUN><NUM>', 39),
 ('<DET><NOUN><NOUN><NOUN><NUM>', 35),
 ('<DET><ADJ><ADJ><NOUN><NOUN><NUM>', 14),
 ('<DET><NOUN><PUNCT><VERB><NOUN><NUM>', 8),
 ('<DET><ADJ><NOUN><NOUN><NUM>', 6),
 ('<DET><ADJ><CCONJ><ADJ><ADJ><NOUN><NOUN><NUM>', 4),
 ('<DET><NOUN><NOUN><NOUN><NOUN><NUM>', 3),
 ('<DET><NOUN><ADP><NOUN><NOUN><NUM>', 3),
 ('<DET><ADJ><CCONJ><ADJ><NOUN><NUM>', 3),
 ('<DET><NOUN><ADP><NOUN><NUM>', 3),
 ('<DET><NOUN><VERB><NOUN><NUM>', 2),
 ('<DET><NOUN><ADV><CCONJ><ADJ><NOUN><NUM>', 1),
 ('<DET><ADJ><VERB><NUM><PUNCT><NUM><ADP><VERB><NOUN><NUM>', 1),
 ('<DET><NOUN><ADP><ADV><VERB><NOUN><NUM>', 1),
 ('<DET><ADV><VERB><NOUN><NUM>', 1),
 ('<DET><ADV><VERB><VERB><NOUN><NUM>', 1),
 ('<DET><VERB><NOUN><NOUN><NUM>', 1),
 ('<DET><NOUN><PUNCT><NOUN><VERB><NOUN><NUM>', 1),
 ('<DET><NOUN><VERB><ADP><VERB><NOUN><NUM>', 1),
 ('<DET><NOUN><ADP><ADJ><ADJ><ADJ><NOUN><NOUN><NUM>', 1),
 ('<DET><ADJ><NOUN><PUNCT><NOUN><PUNCT><VERB><NOUN><NUM>', 1),
 ('<DET><PUNCT><NOUN><PUNCT><NOUN><PUNCT><VERB><NOUN><NUM>', 1),
 ('<DET><VERB><NOUN><ADV><CCONJ><ADJ><NOUN><NUM>', 1),
 ('<DET><NOUN><ADP><ADJ><NOUN><NUM>', 1)]
 ```

Below are some initial functions.

In [2]:
from spacy.symbols import DET, NOUN, CCONJ

def simple_spacy_entity_finder(doc):
    """ Find entities with reference numerals using POS data."""
    entity_list = list()
    record = False
    # Generate a list of tokens so we can iterate backwards through it
    enum_doc_list = list(enumerate(doc))
    last_end = 0
    # Add indices
    for i, word in enum_doc_list:
        if word.pos == DET and not record:
            # Start recording and record start index
            record = True
            start_index = i
        else:        
            if (word.pos == DET or word.pos == CCONJ or word.lemma_ == ";") and record:
                # Step back until last noun is found
                for j, bword in reversed(enum_doc_list[last_end:i]):
                    if bword.pos == NOUN:
                        # Add np_chunk to buffer
                        entity_list.append(doc[start_index:j+1])
                        last_end = j
                        break       
                if word.pos == DET:
                    # Set new start index
                    record = True
                    start_index = i
                else:
                    record = False
    
    entity_dict = dict()
    # Now group by unique
    for entity in entity_list:
        
        np_start = entity.start
        # Ignore the determinant 
        if doc[np_start].pos == DET:
            np_start += 1
        # Generate a string representation excluding the determinant
        np_string = doc[np_start:entity.end].text.lower()
                                
        if np_string not in entity_dict.keys():
            entity_dict[np_string] = list()          
        entity_dict[np_string].append(entity)
    
    return entity_list, entity_dict

In [3]:
from spacy.symbols import DET, NOUN

def check_ant(doc, entity_dict):
    """ Check antecedence - attempt to merge entries with incorrect antecedence."""
    
    issue_keys_a = list()
    issue_keys_the = list()
    
    # Look for entries with antecedence issues
    for key in entity_dict:
        entities = entity_dict[key]
        # Check if first entry begins with "a" - flag if doesn't
        first_entry = entities[0]
        if first_entry[0].pos == DET and first_entry[0].lemma_ != "a" and first_entry[0].lemma_ != "an":
            issue_keys_a.append(key)
        
        # If more than one entry check subsequent entries start with "the" - flag if don't
        if len(entities) > 1:
            for entity in entities[1:]:
                if entity[0].pos == DET and entity[0].lemma_ != "the":
                    issue_keys_the.append(key)
    
    return issue_keys_a, issue_keys_the
        

In [4]:
def look_for_existing(doc, entity_dict):
    """ Look for previously existing versions of problem keys."""
    # If more than one entry check subsequent entries start with "the" - flag if don't
    issue_keys_a = list()
    for key in entity_dict:
        entities = entity_dict[key]
        # Check if first entry begins with "a" - flag if doesn't
        first_entry = entities[0]
        if first_entry[0].pos == DET and first_entry[0].lemma_ != "a" and first_entry[0].lemma_ != "an":
            issue_keys_a.append(key)
    
    for pkey in issue_keys_a:
        problem_entities = entity_dict[pkey]
        # i.e. list of two longer oblong spans
        # Can we just work with the key initially?
        for key in entity_dict.keys():
            if len(pkey) > len(key) and key in pkey:
                print(key, pkey)


In [5]:
# We now need to collate and create a set of entities
def get_entity_set(entity_list):
    """ Get a set of unique entity n-grams from a list of entities."""
    ngram_list = list()
    for entity in entity_list:
        ngram_list.append(" ".join([word for word, pos in entity if (pos != 'DET')]))
    return set(ngram_list)

## Testing on Other Patent Data

Lets test on different patent claims.

In [6]:
# Generate or create some test claim sets for analysis

# (Looks like we can't pickle and load spaCy objects)
from patentdata.corpus import USPublications

pubs = USPublications("/media/SAMSUNG1/Patent_Downloads")
filegenerator = pubs.patentdoc_generator(['G', '06'], sample_size=10)
docs = list(filegenerator)
ent_from_claims = list()
nlp_docs = list()
for doc in docs:
    nlp_doc = nlp(doc.claimset.get_claim(1).text)
    entity_list, entity_dict = simple_spacy_entity_finder(nlp_doc)
    nlp_docs.append(nlp_doc)
    ent_from_claims.append(entity_dict) 

554570 records located.
10 records sampled.


In [7]:
ent_from_claims[0]

{'energy collectors generate collector-local information': [the energy collectors generate collector-local information],
 'energy park': [An energy park],
 'information': [the information],
 'plurality of energy collectors having known spatial location': [a plurality of energy collectors having known spatial location]}

In [8]:
ika, ikt = check_ant(nlp_docs[0], ent_from_claims[0])
print("These terms are not explicitly introduced using 'a/an X':\n", ika, "\n")

print("These terms do not use 'the' yet occur previously:\n", ikt, "\n") 

These terms are not explicitly introduced using 'a/an X':
 ['information', 'energy collectors generate collector-local information'] 

These terms do not use 'the' yet occur previously:
 [] 



In [9]:
nlp_docs[0]


1. An energy park including a plurality of energy collectors having known spatial location where the energy collectors generate collector-local information and the information from the plurality of collectors is spatially correlated.

In [10]:
for d, e in zip(nlp_docs, ent_from_claims):
    print(d, "\n")
    print(e, "\n------\n")


1. An energy park including a plurality of energy collectors having known spatial location where the energy collectors generate collector-local information and the information from the plurality of collectors is spatially correlated.
 

{'energy park': [An energy park], 'plurality of energy collectors having known spatial location': [a plurality of energy collectors having known spatial location], 'information': [the information], 'energy collectors generate collector-local information': [the energy collectors generate collector-local information]} 
------


1. A method used for supporting designing of a printed circuit board including a plurality of conductive layers including conductive areas to which a constant potential is applied, comprising:
specifying conductive areas including wiring from the conductive areas for each of the plurality of conductive layers;
extracting areas which overlap each other from the specified conductive areas;
specifying an interlayer connection member 

Observations:
* Matching occurrences of "the X" with other entries looks generally useful (e.g. is needed across multiple claims). Phrases such as "the given X" or "the selected X" also appear.
* There are some long sections that appear not to meet the simple parse.
* Some have a blank entity?
* We could use the noun_chunks as a second test and merge for greater accuracy?
* Doesn't work so well on some method claims.
* Need to stop on punctuation as well, i.e. "," or ";"
* "said" needs to be a DET.
* Plurals cause an issue, e.g. "multimedia data"

In [11]:
for d, e in zip(nlp_docs, ent_from_claims):
    print("----\n", list(d.noun_chunks), "\n")
    print(list(e.keys()))
    print("\n-----\n")

----
 [An energy park, a plurality, energy collectors, known spatial location, the energy collectors, collector-local information, the information, the plurality, collectors] 

['energy park', 'plurality of energy collectors having known spatial location', 'information', 'energy collectors generate collector-local information']

-----

----
 [A method, designing, a printed circuit board, a plurality, conductive layers, conductive areas, a constant potential, conductive areas, wiring, areas, the plurality, conductive layers, areas, the specified conductive areas, an interlayer connection member, the plurality, conductive layers, the extracted area, an area, the extracted areas, a predetermined distance, the specified interlayer connection member] 

['printed circuit board', '', 'method used for supporting designing', 'conductive areas', 'constant potential is applied, comprising:\nspecifying conductive areas including wiring', 'area', 'plurality of conductive layers', 'predetermined dis

Can I define the problem using probabilities?  

Entities are latent variables of which the words are the visible / observable data.  

Problem is aligning groups of tokens with entities. Classification in a case where we don't know what the classes are or how many classes there are.  

P(entity | words)

What do we know for certain:
* It will have a form of DET ... NOUN or no DET but noun phrase ending in NNS

In [12]:
def annotated_entity_extraction(doc):
    entity_list = list()
    record = False
    # Generate a list of tokens so we can iterate backwards through it
    enum_doc_list = list(enumerate(doc))
    last_end = 0
    # Add indices
    for i, word in enum_doc_list:
        print(i, word, record)
        if word.pos == DET and not record:
            # Start recording and record start index
            record = True
            start_index = i
            print("Starting to record at {0}-{1}".format(i, word))
        else:        
            if (word.pos == DET or word.pos == CCONJ or word.lemma_ == ";" or word.lemma_ == '.') and record:
                print("Stepping back at {0}-{1}".format(i, word))
                # Step back until last noun is found
                added = False
                for j, bword in reversed(enum_doc_list[last_end:i]):
                    print(j, bword, last_end)
                    if bword.pos == NOUN:
                        # Add np_chunk to buffer
                        print("-----> Adding from {0}-{1} = {2}".format(j, i, doc[start_index:j+1]))
                        entity_list.append(doc[start_index:j+1])
                        last_end = j+1
                        added = True
                        break
                # Here if nothing has been added, e.g. no noun found, we need to keep recording
                if word.pos == DET:
                    # Set new start index
                    record = True
                    start_index = i
                    print("Starting to record again at {0}-{1}".format(i, word))
                else:
                    if (word.pos == CCONJ and not added):
                        record = True
                    else:
                        record = False

In [13]:
annotated_entity_extraction(nlp_docs[0])

0 
 False
1 1 False
2 . False
3 An False
Starting to record at 3-An
4 energy True
5 park True
6 including True
7 a True
Stepping back at 7-a
6 including 0
5 park 0
-----> Adding from 5-7 = An energy park
Starting to record again at 7-a
8 plurality True
9 of True
10 energy True
11 collectors True
12 having True
13 known True
14 spatial True
15 location True
16 where True
17 the True
Stepping back at 17-the
16 where 6
15 location 6
-----> Adding from 15-17 = a plurality of energy collectors having known spatial location
Starting to record again at 17-the
18 energy True
19 collectors True
20 generate True
21 collector True
22 - True
23 local True
24 information True
25 and True
Stepping back at 25-and
24 information 16
-----> Adding from 24-25 = the energy collectors generate collector-local information
26 the False
Starting to record at 26-the
27 information True
28 from True
29 the True
Stepping back at 29-the
28 from 25
27 information 25
-----> Adding from 27-29 = the information
Start

In [14]:
annotated_entity_extraction(nlp_docs[1])

0 
 False
1 1 False
2 . False
3 A False
Starting to record at 3-A
4 method True
5 used True
6 for True
7 supporting True
8 designing True
9 of True
10 a True
Stepping back at 10-a
9 of 0
8 designing 0
-----> Adding from 8-10 = A method used for supporting designing
Starting to record again at 10-a
11 printed True
12 circuit True
13 board True
14 including True
15 a True
Stepping back at 15-a
14 including 9
13 board 9
-----> Adding from 13-15 = a printed circuit board
Starting to record again at 15-a
16 plurality True
17 of True
18 conductive True
19 layers True
20 including True
21 conductive True
22 areas True
23 to True
24 which True
25 a True
Stepping back at 25-a
24 which 14
23 to 14
22 areas 14
-----> Adding from 22-25 = a plurality of conductive layers including conductive areas
Starting to record again at 25-a
26 constant True
27 potential True
28 is True
29 applied True
30 , True
31 comprising True
32 : True
33 
 True
34 specifying True
35 conductive True
36 areas True
37 inclu

In [15]:
def np_entity_finder(doc):
    """ Find entities using noun phrases/chunks."""
    entity_dict = dict()
    for entity in doc.noun_chunks:
        np_start = entity.start
        # Ignore the determinant 
        if doc[np_start].pos == DET:
            np_start += 1
        # Generate a string representation excluding the determinant
        np_string = doc[np_start:entity.end].text.lower()
                                
        if np_string not in entity_dict.keys():
            entity_dict[np_string] = list()          
        entity_dict[np_string].append(entity)
        
    return entity_dict

In [16]:
np_entity_finder(nlp_docs[1])

{'area': [an area],
 'areas': [areas, areas],
 'conductive areas': [conductive areas, conductive areas],
 'conductive layers': [conductive layers,
  conductive layers,
  conductive layers],
 'constant potential': [a constant potential],
 'designing': [designing],
 'extracted area': [the extracted area],
 'extracted areas': [the extracted areas],
 'interlayer connection member': [an interlayer connection member],
 'method': [A method],
 'plurality': [a plurality, the plurality, the plurality],
 'predetermined distance': [a predetermined distance],
 'printed circuit board': [a printed circuit board],
 'specified conductive areas': [the specified conductive areas],
 'specified interlayer connection member': [the specified interlayer connection member],
 'wiring': [wiring]}

In [17]:
simple_spacy_entity_finder(nlp_docs[1])

([A method used for supporting designing,
  a printed circuit board,
  a plurality of conductive layers including conductive areas,
  a constant potential is applied, comprising:
  specifying conductive areas including wiring,
  the conductive areas,
  ,
  the plurality of conductive layers,
  ,
  the specified conductive areas,
  an interlayer connection member,
  the plurality of conductive layers,
  the extracted area,
  an area,
  the extracted areas,
  a predetermined distance],
 {'': [, ],
  'area': [an area],
  'conductive areas': [the conductive areas],
  'constant potential is applied, comprising:\nspecifying conductive areas including wiring': [a constant potential is applied, comprising:
   specifying conductive areas including wiring],
  'extracted area': [the extracted area],
  'extracted areas': [the extracted areas],
  'interlayer connection member': [an interlayer connection member],
  'method used for supporting designing': [A method used for supporting designing],
  '

In [18]:
np_entity_finder(nlp_docs[2])

{'current session': [a current session, the current session],
 'file information': [file information],
 'file system information': [file system information],
 'former session': [a former session],
 'information': [information],
 'metadata file': [a metadata file],
 'step': [the step],
 'write-once optical disc': [a write-once optical disc,
  the write-once optical disc]}

In [19]:
simple_spacy_entity_finder(nlp_docs[2])

([a write-once optical disc,
  a file system information recording method,
  the write-once optical disc,
  the step of recording file information,
  a former session,
  a current session,
  a metadata file],
 {'current session': [a current session],
  'file system information recording method': [a file system information recording method],
  'former session': [a former session],
  'metadata file': [a metadata file],
  'step of recording file information': [the step of recording file information],
  'write-once optical disc': [a write-once optical disc,
   the write-once optical disc]})

## Improving the Algorithm

What do we know:
* A DET or a NOUN will always form part of an entity.
* A plural noun may not start with a DET.
* An entity will consist of consecutive tokens.
* The world following a DET will be part of the entity.
* Each determinant can only be linked to one of the nouns in front of it before the next determinant or [";", ":", "."] (and possibly ",").
* Entities with a "the" determinant should have occurred before.
* There are no overlaps.
* We can be more confident if a phrase is repeated.
* We can be more confident still if the phrase is repeated that initially starts with "a" and the next occurrence starts with "the" or "said".
* "said" should be taken as a DET.
* There will be between 1 and number of NOUNS entities.
* The boundary of an entity will be marked by NOUN NOTNOUN - however this pattern can also occur as part of the noun phrase for the entity.
* Entity text sequences will not cross a ":" or ";".
* Occurrences of an entity will have matching text including at least a matching noun.

Definite constraints for a well-formed claim:
* A NOUN will always form part of an entity;
* A singular noun will have a determinant;
* An entity will consist of consecutive tokens.
* There are no overlaps in occurrences - a word can only be linked to a single entity.
* There will be between 1 and number of NOUNS entities.
* Entity text sequences will not cross a ":" or ";" or "." (and possibly a ",").
* The boundary of an entity will be marked by NOUN NOTNOUN - however this pattern can also occur as part of the noun phrase for the entity.

We want to calculate the probability of a set of entities, $ \boldsymbol E $, given a claim as a sequence of words, $ \boldsymbol W $: $$ P(\boldsymbol E | \boldsymbol W) $$   

In fact we want to calculate: $$ \underset{\boldsymbol E}{\operatorname{argmax}} P(\boldsymbol E | \boldsymbol W) $$

Our claim has a length $ N $:$$\boldsymbol W = (\boldsymbol w_0, \boldsymbol w_1, ..., \boldsymbol w_{N})$$

$N$ may be calculated as the length of the claim in tokens.

Each word $\boldsymbol w_i$ has:
* text - $t_i$;
* a simple POS tag - $pos_i$;
* a more detailed POS tag - $posplus_i$;
* a lemma (i.e. a normalised word form) - $lemma_i$; and
* dependeny tree information - $dep_i$.

I.e. $$ \boldsymbol w_i = (t_i, pos_i, posplus_i, lemma_i, dep_i) $$

We have $ M $ entities: $$\boldsymbol E = (e_0, e_1, ..., e_{M})$$ 

where $\boldsymbol e_0 $ indicates "no related entity" or a "null" token. $M$ is not known but will be greater than 2 and less than a number of nouns.

An occurrence is a set of consecutive tokens: $$ \boldsymbol o_k = [\boldsymbol w_i, \boldsymbol w_{i+1}, ..., \boldsymbol w_{i+L_{k}}] $$ where $L_k$ is the length of occurrence $k$ which begins at word index $i$.

$$ \boldsymbol W = [o_1, o_2, ..., o_K] $$ where there are $K$ total occurrences in the claim. However, we don't know $K$ for sure. 

We do know the number of nouns $N_{noun}$. And we know $1 \leqslant K \leqslant N_{noun}$. Also $M \leqslant K$

An entity can have:
* a set of one or more occurrences;
* a string representation - possibly equal to common text across the set of occurrences;
* a number (e.g. be singular or plural).

An entity may be though of as a class label that is applied to a word: $$ \sum_{i=0}^M p(e_i | w) = 1 $$

We know that $ p(e_0 | pos = {DET}) = p(e_0 | pos = {NOUN}) = 0 $, i.e. that determinants and nouns will be assigned to some entity. We also know $ p(e_0 | t = ";") = p(e_0 | t = ":") = p(e_0 | t = ".") = 1$.

Entities are primarily just groupings of word spans, wherein the grouping creates a discrete entity?


$$ \sum_{i=0}^M P(\boldsymbol o_k | e_i) = 1$$

Decomposing using Bayes' Rule: 

$$ \underset{\boldsymbol e}{\operatorname{argmax}} P(\boldsymbol e | \boldsymbol w) = {P(\boldsymbol w | \boldsymbol e) P(\boldsymbol e)}/ P(\boldsymbol w)$$ 

where we can ignore the denominator as we are looking for argmax: $$ \underset{\boldsymbol e}{\operatorname{argmax}} P(\boldsymbol e | \boldsymbol w) = {P(\boldsymbol w | \boldsymbol e) P(\boldsymbol e)}$$

In other models $P(\boldsymbol w | \boldsymbol e)$ and $P(\boldsymbol e)$ may be approximated by a product of transitions (e.g. as per a hidden markov model). However, we have dependencies across sets of words.

Each determinant can only be linked to one of the nouns in front of it before the next determinant.

Start by setting each noun as a separate entity? And marking the tokens that are not an entity? Or look at confident selections e.g. DET NOUN [:;.,]

We can maybe start with a binary classification: $\boldsymbol e = [0,1]$? No, we can confidently apply a positive determination but our negative determination is unknown, i.e. a word that is not positively marked may still form part of an entity.

We can estimate $M$ by counting the number of "a"/"an" determinants + the number of multiple nouns.  

Issue multiple nouns are often introduced by "a X of Ys".  

Also we have "at least one X" and "one or more Ys" - these may not be introduced by "a" or "an" and "at least one" may be referred to again as "the at least one".  

Can we use an estimate of number of determinants as a lower bound?

This works fairly well for a lower bound / initial estimate.  

We can cross check later for missing plural nouns.

How do we model a sequential constraint? 

For each word $w_i$

In [43]:
# This is our good algorithm

# Start with all words relate to no entities
p_all_e_word = dict()

def check_start_phrase(token, doc):
    """ Check for start of phrases 'at least one' and 'one or more' as determinant.
    
    Return true if located."""
    i = token.i
    condition = (
        doc[i:i+3].text.lower() == "at least one" or
        doc[i:i+3].text.lower() == "one or more"
    )
    condition = condition and (doc[i-1].text.lower() != "the")
    return condition

def is_det(token, doc):
    """ Wrapper function for determinant check."""
    # Add 'said' as custom determination
    condition = (token.pos == DET or token.text == "said")
    # Alternatively we can have the start phrases as above
    condition = (condition or check_start_phrase(token, doc))
    # Add check for 'a)' and 'a.' - this is not a det
    condition = condition and (doc[token.i:token.i+2].text.lower() not in ['a)', 'a.'])
    return condition

# 6 is a good test claim
doc = nlp_docs[4]
noun_count = list()

# Initialise probabilities
for token in doc:
    p_all_e_word[token] = dict()

# First parse of tokens
for token in doc:
    p_all_e_word[token] = dict()
    if is_det(token, doc):
        p_all_e_word[token][0] = 0
        # Also set next word as an entity
        p_all_e_word[doc[token.i+1]][0] = 0
    elif token.pos == NOUN:
        p_all_e_word[token][0] = 0
        noun_count.append(token)
    if token.text in [":",";",".", ","]:
        p_all_e_word[token][0] = 1

print("First pass")
for token in doc:
    print(token.text, "[{0}]".format(p_all_e_word[token]), end = '\n') 
#print("Number of nouns = {0}".format(len(noun_count)))

print("Second pass")
# Second parse to fill in probabilities given hard end points
last_break = 0
for token in doc:
    # Look for hard end points
    if p_all_e_word[token].get(0, None) == 1:
        # Look at previous token
        previous_word = doc[token.i - 1]
        # If it is set as an entity (i.e. e_0=0)
        if p_all_e_word[previous_word].get(0, None) == 0:
            print("{0} is e_0=0".format(previous_word))
            # Go back to next e_0=0 entry
            for j in range(token.i-2, last_break, -1):
                if p_all_e_word[doc[j]].get(0, None) == 0:
                    # If last e_0=0 entry is a determinant
                    print("Next - {0} is e_0=0".format(doc[j]))
                    if is_det(doc[j], doc):
                        print("Found - {0}".format(doc[j:token.i]))
                        # Set in between tokens as e_0=0
                        for k in range(j+1, token.i):
                            p_all_e_word[doc[k]][0] = 0 
                    # Finish when hitting previous e_0 token
                    break

                    
for token in doc:
    print(token.text, "[{0}]".format(p_all_e_word[token]), end = '\n') 
#print("Number of nouns = {0}".format(len(noun_count)))

print("Third pass") 
# Third parse - take any DET ... NOUN <boundary> portions
last_break = 0
spans_to_match = list()
for token in doc:
    # Look for hard end points
    if p_all_e_word[token].get(0, None) == 1:
        print("{0} is e_0=1 - looking back".format(token))
        # See if there is a continuous set of e_0 = 0 ending with a noun and starting with a DET
        for j in range(token.i-1, last_break, -1):
            # Look for e_0=0 and noun (do we need to limit to singular noun)
            if p_all_e_word[doc[j]].get(0, None) == 0 and doc[j].pos == NOUN:
                print("Next - {0} is e_0=0 and noun".format(doc[j]))
                # Look back for DET
                for k in range(j, last_break, -1):
                    if p_all_e_word[doc[k]].get(0, None) != 0:
                        # Exit if don't meet a e_0 token
                        break
                    elif doc[k].pos == DET:
                        print("Next - {0} is e_0=0 and DET".format(doc[k]))
                        print("Last break set to {0}".format(token.i))
                        spans_to_match.append((k,j+1))
                        last_break = token.i
                        break
                break
                
print("\n--------\n")

entity_dict = dict()
for stm in spans_to_match:
    print("Looking for matches for '{0}'".format(doc[stm[0]:stm[1]]))
    non_det_string = doc[stm[0]+1:stm[1]].text
    if non_det_string not in entity_dict.keys():
        entity_dict[non_det_string] = list()
    entity_dict[non_det_string].append(doc[stm[0]:stm[1]])
    
print("Unique entities include {0}".format(list(entity_dict.keys()))) 

#Add entity values
entity_count = 0
for entity_string in entity_dict.keys():
    entity_occurrences = entity_dict[entity_string]
    entity_count += 1
    print(entity_dict[entity_string])
    for occurrence in entity_occurrences:
        for token in occurrence:
            p_all_e_word[token][entity_count] = 1
        
for token in doc:
    print(token.text, "[{0}]".format(p_all_e_word[token]), end = '\n') 

# Compare with existing methods 
print(np_entity_finder(doc))
print(simple_spacy_entity_finder(doc))

# Here look for matches that are found with both methods that have an ["a ...","the ...", ...] pattern 

First pass

 [{}]
1 [{}]
. [{0: 1}]
A [{0: 0}]
device [{0: 0}]
, [{0: 1}]
comprising [{}]
: [{0: 1}]

 [{}]
at [{0: 0}]
least [{}]
one [{}]
encrypted [{}]
processor [{0: 0}]
that [{}]
is [{}]
configured [{}]
to [{}]
determine [{}]
an [{0: 0}]
input [{0: 0}]
map [{0: 0}]
associated [{}]
with [{}]
a [{0: 0}]
texture [{0: 0}]
pattern [{0: 0}]
and [{}]
calculate [{}]
a [{0: 0}]
reduced [{}]
resolution [{0: 0}]
pattern [{0: 0}]
based [{}]
on [{}]
the [{0: 0}]
input [{0: 0}]
map [{0: 0}]
; [{0: 1}]
and [{}]

 [{}]
at [{0: 0}]
least [{}]
one [{}]
second [{}]
processor [{0: 0}]
, [{0: 1}]
communicably [{}]
connected [{}]
to [{}]
the [{0: 0}]
at [{}]
least [{}]
one [{}]
encrypted [{}]
processor [{0: 0}]
, [{0: 1}]
that [{0: 0}]
is [{}]
configured [{}]
to [{}]
: [{0: 1}]

 [{}]
receive [{}]
the [{0: 0}]
reduced [{}]
resolution [{0: 0}]
pattern [{0: 0}]
and [{}]
a [{0: 0}]
plurality [{0: 0}]
of [{}]
similar [{}]
patterns [{0: 0}]
, [{0: 1}]
each [{0: 0}]
associated [{}]
with [{}]
a [{0: 0}]
store

Heuristics:
* "for" marks a non-entity [e_0=1]
* "DET X of ..." [e_0=0]
* "in X with" [e_0=1]
* "at least one" / "one or more" [e_0=0]
* lemma = \["comprise", "have", "be", "include"\] [e_0=1]
* "where" in token.text [e_0=1] (e.g. "where or wherein")
* "associated with" [e_0=1]
* "configured/adapted to" [e_0=0]

Also watch out for "each of the plurality of X" or "at least one of the plurality of X"

In [103]:
from spacy.symbols import NUM



# We want to set these if they are not already set
def set_probability(token, p_all_e_word, entity, new_value):
    """ Set probability value if not set already"""
    if entity not in p_all_e_word[token].keys():
        if sum([v for k, v in p_all_e_word[token]] + new_value) <= 1: 
            p_all_e_word[token][entity] = new_value
    return p_all_e_word
            

def heuristics(token, doc, p_all_e_word):
    """ Apply heuristics to mark entity probabilities"""
    entity_stop_chars = ["\n",":",";",".", ","]
    # Set stop characters as non-entity
    if token.text in entity_stop_chars:
        p_all_e_word[token][0] = 1
    
    # Set noun as entity
    if token.pos == NOUN:
        p_all_e_word[token][0] = 0
    
    # 'for' is an entity boundary
    if token.lemma_ == "for":
        p_all_e_word[token][0] = 1
    
    # "comprise", "have", "be", "include" do not relate to an entity
    if token.lemma_ in ["comprise", "have", "be", "include"]:
        p_all_e_word[token][0] = 1
    
    # "where" and "wherein" do not relate to an entity
    if "where" in token.lemma_:
         p_all_e_word[token][0] = 1
    
    # Look ahead - check not at end
    if token.i < (len(doc)-1):
        
        # "configured/adapted to" do not relate to an entity
        if doc[token.i+1].lemma_ == "to" and token.lemma_ in ["configure", "adapt"]:
            p_all_e_word[token][0] = 1
            p_all_e_word[doc[token.i + 1]][0] = 1
    
    if token.i < (len(doc)-2):
        # Set DETs as entity
        if (
            token.pos == DET or token.text == "said"
        ) and (
            doc[token.i:token.i+2].text.lower() not in ['a)', 'a.']
        ):
            p_all_e_word[token][0] = 0
            p_all_e_word[doc[token.i+1]][0] = 0
            
        # DET X of .. relates to an entity
        if token.pos == DET and doc[token.i+2].lemma_ == "of":
            p_all_e_word[token][0] = 0
            p_all_e_word[doc[token.i + 1]][0] = 0
            # Set of
            p_all_e_word[doc[token.i + 2]][0] = 0
            # Set term after off
            p_all_e_word[doc[token.i + 3]][0] = 0
            
        # "in X with" does not relate to an entity
        if token.lemma_ == "in" and doc[token.i+2].lemma_ == "with":
            p_all_e_word[token][0] = 1
            p_all_e_word[doc[token.i + 1]][0] = 1
            p_all_e_word[doc[token.i + 2]][0] = 1
            
        # Associated with does not relate to an entity
        if doc[token.i:token.i+2].text.lower() == "associated with":
            p_all_e_word[token][0] = 1
            p_all_e_word[doc[token.i + 1]][0] = 1
    
    if token.i < (len(doc)-3):
        # "at least NUM" / "NUM or more" relates to an entity
        if doc[token.i:token.i + 2].text.lower() == "at least" and doc[token.i + 2].pos == NUM:
            p_all_e_word[token][0] = 0
            p_all_e_word[doc[token.i + 1]][0] = 0
            p_all_e_word[doc[token.i + 2]][0] = 0
        if doc[token.i+1:token.i + 3].text.lower() == "or more" and token.pos == NUM:
            p_all_e_word[token][0] = 0
            p_all_e_word[doc[token.i + 1]][0] = 0
            p_all_e_word[doc[token.i + 2]][0] = 0
    
    return p_all_e_word
    

In [134]:
from collections import OrderedDict
# Try out with adding heuristics

doc = nlp_docs[4]

# Start with all words relate to no entities
p_all_e_word = dict()

# Initialise probabilities
for token in doc:
    p_all_e_word[token] = dict()

# Is the order of our labelling important? Probably as we overwrite following probs    
    
# This can be combined with first pass easily - similar checks
print("First pass - entity label heuristics")
for token in doc:
    p_all_e_word = heuristics(token, doc, p_all_e_word)
    print(token.text, "[{0}]".format(p_all_e_word[token]), end = '\n')
   
print("Second pass - look for DET ... NOUN groupings") 
# Third parse - take any DET ... NOUN <boundary> portions
last_break = 0
spans_to_match = list()
for token in doc:
    # Look for hard end points or DET
    if (p_all_e_word[token].get(0, None) == 1) or (token.pos == DET):
        print("{0} is e_0=1 or DET - looking back".format(token))
        # Step back marking as e_0=1 until first NOUN      
        for j in range(token.i-1, last_break, -1):
            print("Step back token - {0} with pos - {1}".format(doc[j], doc[j].pos))
            if doc[j].pos != NOUN:
                print("Setting non-Noun")
                p_all_e_word[doc[j]][0] = 1
            else:
                last_break = j
                break
    # Look at grouping from DET
    if is_det(token, doc):
        # Tweak for "at least X" and "X or more"
        if (
            doc[token.i:token.i + 2].text.lower() == "at least" and doc[token.i + 2].pos == NUM
        ) or (
            doc[token.i+1:token.i + 3].text.lower() == "or more" and token.pos == NUM
        ):
            #print("Head index set to {0}".format())
            head_index = doc[token.i+2].head.i
        else: 
            head_index = token.head.i
        possible_entity = True
        # Step through intermediate tokens between current and head
        for j in range(token.i, head_index):
            # If head is outside of DET ... end_NOUN sequence
            if doc[j].head.i < token.i and doc[j].head.i > head_index:
                # Check for nested portions
                possible_entity = False
        if possible_entity:
            for k in range(token.i, head_index + 1):
                p_all_e_word[doc[k]][0] = 0 
  # Need to adapt the above for at least one ... X and one or more ... Xs - "at" > head > "least" > "one" > X              
    
for token in doc:
    print(token.text, "[{0}]".format(p_all_e_word[token]), end = '\n') 

print("Extracted possible occurrences:\n")
poss_occ = list()
for token in doc[1:]:
    # If transition
    if p_all_e_word[token].get(0, 1) == 0 and p_all_e_word[doc[token.i-1]].get(0, 0) == 1:
        # Add consecutive e_0=0
        for j in range(token.i, len(doc)+1):
            if p_all_e_word[doc[j]].get(0, 1) != 0:
                poss_occ.append(doc[token.i:j])
                break

print(poss_occ)

# Matching occurrences
entity_dict = dict()
# Now group by unique
for entity in poss_occ:
    np_start = entity.start
    # Ignore the determinant 
    if doc[np_start].pos == DET:
        np_start += 1
    # Generate a string representation excluding the determinant
    np_string = doc[np_start:entity.end].text.lower()                        
    if np_string not in entity_dict.keys():
        entity_dict[np_string] = list()          
    entity_dict[np_string].append(entity)

print(doc)
# print(entity_dict)

# Quick function to sort entities by occurrence
# Need to sort the keys by the index of the first word in the first entry
ordered_entities = OrderedDict(sorted(entity_dict.items(), key=lambda t: t[1][0][0].i))

print(ordered_entities)


First pass - entity label heuristics

 [{0: 1}]
1 [{}]
. [{0: 1}]
A [{0: 0}]
device [{0: 0}]
, [{0: 1}]
comprising [{0: 1}]
: [{0: 1}]

 [{0: 1}]
at [{0: 0}]
least [{0: 0}]
one [{0: 0}]
encrypted [{}]
processor [{0: 0}]
that [{}]
is [{0: 1}]
configured [{0: 1}]
to [{0: 1}]
determine [{}]
an [{0: 0}]
input [{0: 0}]
map [{0: 0}]
associated [{0: 1}]
with [{0: 1}]
a [{0: 0}]
texture [{0: 0}]
pattern [{0: 0}]
and [{}]
calculate [{}]
a [{0: 0}]
reduced [{0: 0}]
resolution [{0: 0}]
pattern [{0: 0}]
based [{}]
on [{}]
the [{0: 0}]
input [{0: 0}]
map [{0: 0}]
; [{0: 1}]
and [{}]

 [{0: 1}]
at [{0: 0}]
least [{0: 0}]
one [{0: 0}]
second [{}]
processor [{0: 0}]
, [{0: 1}]
communicably [{}]
connected [{}]
to [{}]
the [{0: 0}]
at [{0: 0}]
least [{0: 0}]
one [{0: 0}]
encrypted [{}]
processor [{0: 0}]
, [{0: 1}]
that [{0: 0}]
is [{0: 1}]
configured [{0: 1}]
to [{0: 1}]
: [{0: 1}]

 [{0: 1}]
receive [{}]
the [{0: 0}]
reduced [{0: 0}]
resolution [{0: 0}]
pattern [{0: 0}]
and [{}]
a [{0: 0}]
plurality [

This paper - http://cogprints.org/5025/1/nrc-48727.pdf - suggests a two-phase process:
* Generate a "gazetteer" (a list of named entities) - similar to our first stage of simple_entity_extraction method;
* Disambiguate names in "gazetteer" (this is similar to our second stage of simple_entity_extraction method).

Is the algorithm generally:
* Mark as entity or not based on rules;
* Look back from DET or punct break [':',';',',','.'] - set as non-entity until noun is found;
* Look at noun phrase chunks 

In [ ]:
# Look at plural nouns
if token.tag_ == "NNS":
    #Step back and mark as e_0=0 any preceding word that has the token as a head
    for j in range(token.i, 0, -1):
        if (doc[j].head.i == (doc[j].i-1)) and p_all_e_word[doc[j-1]].get(0,0) != 1:
            p_all_e_word[doc[j-1]][0] = 0

In [130]:
# Look at POS and head for each token
doc = nlp_docs[4]
for token in doc:
    print(token.text, token.i, token.lemma_, token.pos_, token.head.text, token.head.i, token.tag_)


 0 
 SPACE 1 1 SP
1 1 1 PUNCT 1 1 LS
. 2 . PUNCT 1 1 .
A 3 a DET device 4 DT
device 4 device NOUN identify 86 NN
, 5 , PUNCT device 4 ,
comprising 6 comprise VERB device 4 VBG
: 7 : PUNCT comprising 6 :

 8 
 SPACE : 7 SP
at 9 at ADP least 10 IN
least 10 least ADJ one 11 JJS
one 11 one NUM processor 13 CD
encrypted 12 encrypt VERB processor 13 VBN
processor 13 processor NOUN comprising 6 NN
that 14 that ADJ configured 16 WDT
is 15 be VERB configured 16 VBZ
configured 16 configure VERB processor 13 VBN
to 17 to PART determine 18 TO
determine 18 determine VERB configured 16 VB
an 19 an DET map 21 DT
input 20 input NOUN map 21 NN
map 21 map NOUN determine 18 NN
associated 22 associate VERB map 21 VBN
with 23 with ADP associated 22 IN
a 24 a DET pattern 26 DT
texture 25 texture NOUN pattern 26 NN
pattern 26 pattern NOUN with 23 NN
and 27 and CCONJ determine 18 CC
calculate 28 calculate VERB determine 18 VB
a 29 a DET pattern 32 DT
reduced 30 reduced ADJ pattern 32 JJ
resolution 31 resolut

Can we look for each DET then label as entity until head noun if head = head noun?  

If we have a span between an e_0=0 and a noun - label as an entity occurrence if all tokens in span have the same head.

We can get plurals by looking for plural nouns then tracing back to find previous tokens with the plural noun as head.

Patterns like this:
```
a 21 a DET unit 25
vehicle 22 vehicle NOUN side 24
- 23 - PUNCT side 24
side 24 side NOUN unit 25
unit 25 unit NOUN mounted 26
```
Can be resolved - vehicle-side is a subtree where the head of side still points to unit. So the head doesn't need to point to the end noun but it does need.

All our sub-phrases need to be between DET and head noun.

In [ ]:
def check_head(token, doc, end_head):
    """ Check for noun phrase ."""
    if token.head != end_head and token.head.i > token.i:
        check_head(token.head, doc, end_head)
    else:
        if 
        return False

In [29]:
for token in doc:
    i = token.i
    print("'{0}'".format(doc[i:i+3].text.lower()))
    condition = (
        doc[i:i+3].text.lower() == "at least one" or
        doc[i:i+3].text.lower() == "one or more"
    )
    print(token, condition)

'
1.'

 False
'1. a'
1 False
'. a device'
. False
'a device,'
A False
'device, comprising'
device False
', comprising:'
, False
'comprising:
'
comprising False
':
at'
: False
'
at least'

 False
'at least one'
at True
'least one encrypted'
least False
'one encrypted processor'
one False
'encrypted processor that'
encrypted False
'processor that is'
processor False
'that is configured'
that False
'is configured to'
is False
'configured to determine'
configured False
'to determine an'
to False
'determine an input'
determine False
'an input map'
an False
'input map associated'
input False
'map associated with'
map False
'associated with a'
associated False
'with a texture'
with False
'a texture pattern'
a False
'texture pattern and'
texture False
'pattern and calculate'
pattern False
'and calculate a'
and False
'calculate a reduced'
calculate False
'a reduced resolution'
a False
'reduced resolution pattern'
reduced False
'resolution pattern based'
resolution False
'pattern based on'
patte

When matching what to do with 'the time scale' and 'the time scale display information' or 'a project' and:
```
A [{0: 0}]
project [{}]
information [{0: 0}]
display [{0: 0}]
device [{0: 0}]
, [{0: 1}]
comprising [{}]
: [{0: 1}]
```
Only look for e_0 stretches of same number with matching pos and text? (Are we now getting to look at transitions?)

There are issues with "(a)" and "a."

Also "response" from "in response".

Check det is not working for "at least one"

We can iterate back from where e_0 = 1 - tokens between a last noun and determinant will be part of an entity. We can then match those across the claim. This is the simple entity finder but stepping back at [:;,.] as well as DET.  
Pattern is:
* If next step back is e_0=0;
* If next e_0=0 is a check_det=True;
* Fill in inbetween as e_0=0.


In [121]:
np_entity_finder(doc)

{'accordance': [accordance],
 'associated user': [an associated user],
 'communication device': [a communication device,
  the communication device,
  the communication device],
 'content filter engine': [a content filter engine],
 'data': [data],
 'information': [information],
 'personal content filter database': [a personal content filter database],
 'predefined rules': [predefined rules,
  predefined rules,
  predefined rules,
  predefined rules],
 'rules': [rules],
 'server': [a server],
 'set': [a set, the set, the set, the set],
 'system': [A system],
 'user': [user],
 'user interface': [a user interface]}

In [122]:
simple_spacy_entity_finder(doc)[1]

{'associated user': [an associated user],
 'communication device': [the communication device],
 'communication device in accordance': [a communication device in accordance],
 'content filter engine': [a content filter engine],
 'personal content filter database': [a personal content filter database],
 'server': [a server],
 'set of predefined rules': [a set of predefined rules,
  the set of predefined rules],
 'set of predefined rules by preventing restricted information': [the set of predefined rules by preventing restricted information],
 'set of predefined rules comprising user defined rules': [the set of predefined rules comprising user defined rules],
 'system': [the system],
 'system for filtering data': [A system for filtering data],
 'user interface': [a user interface]}

Another pattern is "DET X FOR [phrase]" - this is one entity? But contains references to other entities
```
a [{0: 0}]
system [{0: 0}]
for [{}]
providing [{}]
a [{0: 0}]
plurality [{0: 0}]
of [{}]
football [{0: 0}]
player [{0: 0}]
types [{0: 0}]
from [{}]
which [{}]
a [{0: 0}]
football [{0: 0}]
player [{0: 0}]
type [{0: 0}]
is [{}]
selected 
```

This takes a for clause as the whole entity string - e.g. "A method for modeling electrical characteristics of cells having given circuit elements" and "a layout of cells having at least one cell".